In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# USER DATA

In [2]:
path_users = "/Users/gandalf/Documents/data/data_users.pkl"
user_df = pd.read_pickle(path_users)

# REMOVE NEW WORLD
not_NA_user_df = user_df[(user_df.longitude > -25) | (user_df.latitude < 0)]
not_NA_user_df = not_NA_user_df[user_df.longitude != 0]

not_NA_user_df.head(2)

/Users/gandalf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,created,updated,about,has_about,len_about,available,birthday,age,collegeId,emailVerified,...,petsOk,pictureId,roomPostId,roomTypeId,smokingOk,term,work,sent,responses,ratio
uid,,,,,,,,,,,,,,,,,,,,,
0CotMsEE53,2017-06-26 05:24:44,2018-01-04 21:33:57,,False,0,nan,1983-01-01,35,NaN,1,...,1.0,368.0,NaN,2.0,0.0,12.0,REA,0.0,0.0,0.0
0DjwkULGuT,2017-07-21 15:28:49,2018-01-04 22:23:29,,False,0,nan,1997-01-01,21,NaN,1,...,1.0,384.0,NaN,2.0,1.0,12.0,hotel,0.0,0.0,0.0


In [3]:
outside_NA_uid = not_NA_user_df.index
print(len(outside_NA_uid))

776


In [4]:
path_message = "/Users/gandalf/Documents/data/data_message.pkl"
message_df = pd.read_pickle(path_message)
message_df.head(2)

,message_id,conversation_id,uid,read,readBy,text_length,timestamp,imageURL,emailAttempted,emailed
0,-KdmSRA-1a7niRVp5s83,-KdmSL0izd_44wrniy-W,8Ws3QWmC4B,True,"{'1': '6y3bPICtHM', '0': 'OBFaJBVxWC'}",3,1487979004690,,,
1,-Ke-RBSDV6VsM2fE2cwu,-KdmSL0izd_44wrniy-W,OBFaJBVxWC,True,"{'1': '6y3bPICtHM', '0': '8Ws3QWmC4B'}",3,1488213559164,,,


In [5]:
print("All messages: {}".format(len(message_df)))
outside_NA_messages = message_df[message_df.uid.isin(outside_NA_uid)]
print("Outside NA messages: {}".format(len(outside_NA_messages)))

All messages: 34604
Outside NA messages: 475


In [12]:
outside_NA_messages.head()

,message_id,conversation_id,uid,read,readBy,text_length,timestamp,imageURL,emailAttempted,emailed
130,-Ko2SqogCWPwyXHGkmQb,0k5Et4GYuW0AIXrTs4wJ,0AIXrTs4wJ,False,,1,1499001745026,,True,True
239,-KzMbQ8FsP8sdd2N9Jac,1dEMqt7nXt0FA65LGgpI,1dEMqt7nXt,False,,2,1511150957493,,True,True
240,-KzMbKf1dJz_nDAqS4ow,1dEMqt7nXt0VpnAbRsF0,1dEMqt7nXt,True,,2,1511150935089,,True,True
245,-KsmWCmpRMTvq9TlJTmR,1fpSZU7IVo00xCAIRuOH,1fpSZU7IVo,True,,52,1504086121971,,True,True
254,-KmA-w2V8Exj4EfPswsb,1rSsa83otc01mfWG4vUm,1rSsa83otc,False,,1,1496980898586,,True,True


In [7]:
path_lastmessage = "/Users/gandalf/Documents/data/data_lastmessage.pkl"
lastmessage_df = pd.read_pickle(path_lastmessage)
lastmessage_df.head(2)

,user_id,first_ten,last_ten
lastMessageId,,,
-L-pLJ5Q9ziekprSBl2e,00DMWuKDFs,0sTqXBoFyW,00DMWuKDFs
-KxYXLd8xNLXVxs6hauY,00DMWuKDFs,6MSR8WmfII,00DMWuKDFs


In [8]:
path_response = "/Users/gandalf/Documents/data/data_response.pkl"
response_df = pd.read_pickle(path_response)
response_df.head(5)

,conv_id,first_ten,last_ten,response,first_uid,first_mid,second_uid,second_mid
0,-KdmSL0izd_44wrniy-W,0sTqXBoFyW,00DMWuKDFs,True,8Ws3QWmC4B,-KdmSRA-1a7niRVp5s83,OBFaJBVxWC,-Ke-RBSDV6VsM2fE2cwu
1,-KdmixiTP5parpBtLtBJ,6MSR8WmfII,00DMWuKDFs,True,nQYG2Z9elg,-Kdmj-d4TwKFUMQkPKeF,nQYG2Z9elg,-KdmjBCinEGTfRSdv7-M
2,-Ke18G-UGnLrasv4_xXT,7gfWoOSLZ0,00DMWuKDFs,True,n2ffQeQMri,-Ke18IPzq_R4mOHswlVh,nQYG2Z9elg,-Ke18NcSKBsSu2d4uUCt
3,-Ke28RHZLNV4m69_08sZ,AbWMEZd7XI,00DMWuKDFs,False,nQYG2Z9elg,-Ke28SI_EyKkov9BteUL,None,None
4,-Ke4mWs5IgcfwfRwdRFA,DGcdnJ8260,00DMWuKDFs,True,f5WrJHjy0R,-Ke4mjlJmO0XJUwci1dy,f5WrJHjy0R,-Ke4moue2imO-VAdKdX0


In [9]:
print("All conversations: {}".format(len(response_df)))
outside_NA_conv = response_df[response_df.first_uid.isin(outside_NA_uid)]
print("Outside NA conversations: {}".format(len(outside_NA_conv)))
outside_NA_conv.head()

All conversations: 122396
Outside NA conversations: 337


,conv_id,first_ten,last_ten,response,first_uid,first_mid,second_uid,second_mid
57,0k5Et4GYuW0AIXrTs4wJ,GMoeQ1APAC,00xCAIRuOH,False,0AIXrTs4wJ,-Ko2SqogCWPwyXHGkmQb,None,None
128,1dEMqt7nXt0FA65LGgpI,0Fsuw8S0mL,013LzOrVju,False,1dEMqt7nXt,-KzMbQ8FsP8sdd2N9Jac,None,None
129,1dEMqt7nXt0VpnAbRsF0,9DjJbRPVQ8,013LzOrVju,False,1dEMqt7nXt,-KzMbKf1dJz_nDAqS4ow,None,None
134,1fpSZU7IVo00xCAIRuOH,69tclvEtHT,013LzOrVju,False,1fpSZU7IVo,-KsmWCmpRMTvq9TlJTmR,None,None
140,1rSsa83otc01mfWG4vUm,kKUCCCQbU4,013LzOrVju,False,1rSsa83otc,-KmA-w2V8Exj4EfPswsb,None,None


In [14]:
outside_NA_conv.response.sum()/len(outside_NA_conv)

0.05934718100890208

In [15]:
outside_NA_conv['first_ten'] = outside_NA_conv.conv_id.apply(lambda x: x[:10])
outside_NA_conv['last_ten'] = outside_NA_conv.conv_id.apply(lambda x: x[10:])

/Users/gandalf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/gandalf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
# get recipients
for index, row in outside_NA_conv.iterrows():
    if row.second_uid == None:
        if row.first_ten == row.first_uid: row.second_uid = row.last_ten
        elif row.last_ten == row.first_uid: row.second_uid = row.first_ten
        else: print('uh')

In [18]:
outside_NA_conv[outside_NA_conv.response].head()

,conv_id,first_ten,last_ten,response,first_uid,first_mid,second_uid,second_mid
153,20jHWHiCJr0cP281Y8eL,20jHWHiCJr,0cP281Y8eL,True,20jHWHiCJr,-Kvb9y0E29KE0HXvVyCJ,0cP281Y8eL,-KwqkezOow7clqaFjdPd
704,6gDsAm9G4E02i1Py9KrY,6gDsAm9G4E,02i1Py9KrY,True,6gDsAm9G4E,-KrizR2jPgpbkdhO2X4P,6gDsAm9G4E,-KrizSQwe5Xv8Rg12tRP
3731,JvtdkTzOzG1XfG7vzTqk,JvtdkTzOzG,1XfG7vzTqk,True,JvtdkTzOzG,-Kfxd61ZkqKG8M6np9F_,1XfG7vzTqk,-KgiJfKiOtwh02O0YxBL
3740,JvtdkTzOzG4HvaQKueGT,JvtdkTzOzG,4HvaQKueGT,True,JvtdkTzOzG,-KfxXc9gHIer3cCGe4AM,4HvaQKueGT,-Kfx_HMJLyH5cb1XdFvM
4820,NXvjKjFtk31LD5LZFK0B,NXvjKjFtk3,1LD5LZFK0B,True,NXvjKjFtk3,-KtnFqznI4x3VDRvuADa,1LD5LZFK0B,-KtqvCOV_rjS3gYegOrq


In [19]:
simple_cols = ['first_uid','second_uid','response']
simple_df = outside_NA_conv[simple_cols]
# tf = {True:1, False:0, 1:1, 0:0}
# simple_df.response = simple_df.response.map(tf)
simple_df.head()

,first_uid,second_uid,response
57,0AIXrTs4wJ,0k5Et4GYuW,False
128,1dEMqt7nXt,0FA65LGgpI,False
129,1dEMqt7nXt,0VpnAbRsF0,False
134,1fpSZU7IVo,00xCAIRuOH,False
140,1rSsa83otc,01mfWG4vUm,False


In [20]:
print(simple_df.response.sum())
print(len(simple_df.response))

20
337


In [21]:
len(user_df)

45010

In [22]:
user_df.loc['0cP281Y8eL'].isClean

1.0

In [23]:
simple_df.head(1)

,first_uid,second_uid,response
57,0AIXrTs4wJ,0k5Et4GYuW,False


In [24]:
ad,sg,sr,sc,sn,ss = [],[],[],[],[],[]

for index, row in simple_df.iterrows():
    first = user_df.loc[str(row.first_uid)]
    second = user_df.loc[str(row.second_uid)]
    ad.append(abs(first.age - second.age))
    sg.append(first.gender == second.gender)
    sr.append(first.inRelationship == second.inRelationship)
    sc.append(first.isClean == second.isClean)
    sn.append(first.isNight == second.isNight)
    ss.append(first.isStudent == second.isStudent)

simple_df['age_dif'] = ad
simple_df['same_gender'] = sg
simple_df['same_relate'] = sr
simple_df['same_clean'] = sc
simple_df['same_night'] = sn
simple_df['same_student'] = ss

# filled na at some point, should undo

simple_df.head(2)

/Users/gandalf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/gandalf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/gandalf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas

,first_uid,second_uid,response,age_dif,same_gender,same_relate,same_clean,same_night,same_student
57,0AIXrTs4wJ,0k5Et4GYuW,False,6,True,False,True,False,False
128,1dEMqt7nXt,0FA65LGgpI,False,1,False,True,True,True,False


In [25]:
X = simple_df
y = X.response.map({True:1,False:0})
X = X.drop(['first_uid','second_uid','response'], axis=1)
X.head()
y.sum()

20

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)

# mapper = DataFrameMapper([(X_train.columns, StandardScaler())])
# scale = mapper.fit(X_train)
# X_train = pd.DataFrame(mapper.transform(X_train), index=X_train.index, columns=X_train.columns)
# X_test = pd.DataFrame(mapper.transform(X_test), index=X_test.index, columns=X_test.columns)

In [42]:
X_train.head()

,age_dif,same_gender,same_relate,same_clean,same_night,same_student
7084,0,True,True,True,True,True
14511,1,False,True,True,True,False
3740,0,False,True,False,False,False
1663,0,False,True,False,False,False
4378,5,False,True,False,False,False


In [41]:
model = LogisticRegression()
model.fit(X_train, y_train)
print("Prediction:   {}".format(model.predict(X_test)))
print("Actual:       {}".format(y_test.values))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

Prediction:   [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Actual:       [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 1 0 0 0 0 0 0 0 0 0 0]
Score of model: 0.9411764705882353
Score to beat: 0.9411764705882353


In [48]:
def do_grid_search(X, y):
#     # Get the data from our function above
#     X, y = format_data()

#     # Split it up into our training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X,y)

    # Initalize our model here
    est = GradientBoostingClassifier()

    # Here are the params we are tuning, ie, 
    # if you look in the docs, all of these are 'nobs' within the GradientBoostingClassifier algo. 
    param_grid = {'learning_rate': [0.1, 0.05, 0.02],
                  'max_depth': [2, 3],
                  'min_samples_leaf': [3, 5],
                  }

    # Plug in our model, params dict, and the number of jobs, then .fit()
    gs_cv = GridSearchCV(est, param_grid, n_jobs=2).fit(X_train, y_train)

    # return the best score and the best params
    return gs_cv.best_score_, gs_cv.best_params_



def do_regular_decision_tree(X, y, best_max_depth):
    # Split it up into our training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X,y)

    # Initalize our decision tree algo, set the max_depth to our maximum_max_depth
    clf = DecisionTreeClassifier(max_depth=best_max_depth)
    
    # Fit our tree
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Find the precision, recall, and score
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    score = clf.score(X_test, y_test)
    return precision, recall, score



best_score, best_grid_params =  do_grid_search(X,y)

best_max_depth = best_grid_params['max_depth']

print(do_regular_decision_tree(best_max_depth))

(0.0, 0.0, 1.0)


In [60]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Prediction:   {}".format(model.predict(X_test)))
print("Actual:       {}".format(y_test.values))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("   Of things the model says are 1's, how many are 1's?")
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("   Of things that are 1's, how many does the model says are 1's?")
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

Prediction:   [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0]
Actual:       [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 1 0 0 0 0 0 0 0 0 0 0]
Model recall: 0.2
   Of things the model says are 1's, how many are 1's?
Model precision: 0.125
   Of things that are 1's, how many does the model says are 1's?
Score of model: 0.8705882352941177
Score to beat: 0.9411764705882353


In [61]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
print("Prediction:   {}".format(model.predict(X_test)))
print("Actual:       {}".format(y_test.values))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("   Of things the model says are 1's, how many are 1's?")
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("   Of things that are 1's, how many does the model says are 1's?")
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

Prediction:   [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Actual:       [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 1 0 0 0 0 0 0 0 0 0 0]
Model recall: 0.2
   Of things the model says are 1's, how many are 1's?
Model precision: 0.125
   Of things that are 1's, how many does the model says are 1's?
Score of model: 0.9294117647058824
Score to beat: 0.9411764705882353


In [62]:
model = GradientBoostingClassifier()
param_grid = {'learning_rate': [0.1, 0.05, 0.02],
              'max_depth': [2, 3],
              'min_samples_leaf': [3, 5],
              }
max_depth=
gs_cv = GridSearchCV(est, param_grid, n_jobs=2).fit(X_train, y_train)

model.fit(X_train, y_train,gs_cv.best_params_['max_depth'])
print("Prediction:   {}".format(model.predict(X_test)))
print("Actual:       {}".format(y_test.values))
print("Model recall: {}".format(recall_score(y_test, y_pred)))
print("   Of things the model says are 1's, how many are 1's?")
print("Model precision: {}".format(precision_score(y_test, y_pred)))
print("   Of things that are 1's, how many does the model says are 1's?")
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

Prediction:   [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0]
Actual:       [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 1 0 0 0 0 0 0 0 0 0 0]
Model recall: 0.2
   Of things the model says are 1's, how many are 1's?
Model precision: 0.125
   Of things that are 1's, how many does the model says are 1's?
Score of model: 0.9176470588235294
Score to beat: 0.9411764705882353
